In [164]:
import json
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
#from langchain.prompts import PromptTemplate
from openai import OpenAI
from pathlib import Path
from pydantic import BaseModel, Field
from typing import List, Union


load_dotenv(override=True)
client = OpenAI()

In [2]:
# Set working directory - Not required if using Jupyter outside of VScode
workdir = os.environ["workdir"]
os.chdir(workdir)
os.getcwd()

'/Users/junaid/Developer/student_feedback'

In [4]:
# Load in Data
data_dir = Path("./data/")

student_answers_llm_graded = pd.read_csv(data_dir / "student_answers_llm_graded.csv")

In [5]:
student_answers_llm_graded.head()

,subject_id,question_id,question_type,student_id,question_text,mark_scheme_text,context,answer_text,llm_graded_answer,awarded_marks,...,llm_mark_hitrate,llm_level_hitrate,scaled_error_metric,answer_id,linked_answer_id,topic_id,answer_scanned_image,elapsed_time_in_seconds,llm_graded_answer_token_costing,llm_awarded_marks_token_costing
0,edexcel_business_studies,1.1,bs_mcq,4133,Which one of the following could market mappin...,\ncorrect_answer:\n - C\ntotal_marks:\n - 1,NaN,C,"The Answer provided is ""C,"" which matches the ...",1,...,True,NaN,NaN,NaN,NaN,NaN,NaN,1.778014,"{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt...","{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt..."
1,edexcel_business_studies,1.2,bs_mcq,4133,Which one of the following is an impact of a d...,correct_answer:\n - C\ntotal_marks:\n - 1,NaN,D,"The Answer provided is ""D,"" which is not the c...",0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,1.224670,"{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt...","{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt..."
2,edexcel_business_studies,1.3,bs_explain,4133,Explain one advantage of starting a business a...,marking_criteria:\n - Identification of an ad...,NaN,You deal and finance with the problems directly,This question requires the candidate to 'Expla...,0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,3.976895,"{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt...","{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt..."
3,edexcel_business_studies,1.4,bs_explain,4133,Explain one way the internet may affect where ...,marking_criteria:\n - Identification of a way...,NaN,A smaller business means a lower reputation so...,This question requires the candidate to 'Expla...,0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,4.320457,"{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt...","{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt..."
4,edexcel_business_studies,2.1,bs_mcq,4133,Which two of the following are examples of cus...,correct_answer:\n - A\n - C\ntotal_marks:\n ...,NaN,"A, C","The Answer ""A, C"" correctly identifies the two...",2,...,True,NaN,NaN,NaN,NaN,NaN,NaN,1.076180,"{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt...","{'burn_in_runs': 1, 'model': 'gpt-4o', 'prompt..."


# Expand Existing Student Feedback

In [136]:
student_answers = student_answers_llm_graded[(student_answers_llm_graded.question_type == "hs_explain") & (student_answers_llm_graded.llm_mark_hitrate == True)].to_dict(orient="records")
student_answers

[{'subject_id': 'aqa_history',
  'question_id': 1.0,
  'question_type': 'hs_explain',
  'student_id': 5008,
  'question_text': 'Source A is critical of the Bolsheviks during the First World War. How do you know? Explain your answer using Source A and your contextual knowledge.',
  'mark_scheme_text': "Level_2:\n  description: \n    - Developed analysis of source based on content and/or provenance\n  marks_awarded:\n    maximum: 4\n    minimum: 3\n  criteria:\n    - Developed analysis of source based on content and/or provenance\n  progression:\n    - Students may progress from a simple analysis of the source with extended reasoning supported by factual knowledge and understanding related to the features of the source.\n  example_answer: \n    - 'The cartoon is British and the title refers to being betrayed by Russia who was one of their allies. The Bolshevik Revolution occurred in 1917 and Lenin made peace with Germany in 1918. This reduced the demands on the German army in the East an

In [138]:
student_answer = [i for i in student_answers if i["student_id"] == 5200][0]
student_answer

{'subject_id': 'aqa_history',
 'question_id': 2.0,
 'question_type': 'hs_explain',
 'student_id': 5200,
 'question_text': 'How useful are Sources B and C to an historian studying military technology and tactics in the First World War? Explain your answer using Sources B and C and your contextual knowledge.',
 'mark_scheme_text': "Level_4:\n  description: \n    - Complex evaluation of both sources with sustained judgment based on content and provenance\n  marks_awarded:\n    maximum: 12\n    minimum: 10\n  criteria:\n    - Students may progress from a developed evaluation of the sources by complex reasoning related to utility on the basis of content and provenance.\n    - They may evaluate the relationship between the sources based on analysis of provenance and contextual knowledge.\n  progression:\n    - Students may progress to this level through complex reasoning related to utility on the basis of content and provenance.\n  example_answer: \n    - 'The sources show that military tech

In [109]:
full_marks_answer = {
    "question_text": student_answer["question_text"],
    "answer_text": "The Alliance System played a significant role in the lead-up to World War I, creating opposing camps in Europe that heightened tensions and fears of encirclement. For example, the Triple Alliance between Germany, Austria-Hungary, and Italy sought to isolate France, while the Triple Entente of Britain, France, and Russia countered this. The rigid structure made diplomacy difficult, and when the assassination of Archduke Franz Ferdinand occurred, it triggered a conflict that rapidly involved all the major powers due to these alliances. Therefore, while the Alliance System was a critical factor, it was intertwined with other issues such as nationalism and imperial rivalries that also contributed to the onset of war.",
    "mark_scheme_text": student_answer["mark_scheme_text"],
    "llm_graded_answer": "This answer gains 16 out of 16 marks"
}

In [139]:
student_feedback_extract_prompt = """
You will be given a Question, Mark Scheme, Student Answer and a Graded Answer on that answer.
The Graded Answer has feedback for how a student can improve their answer.
Your task is to reorientate and expand the Graded Answer to make the student the subject for the advice, you speak in a straightforward, clear and supportive way to the student and offer them specific advice on how their answer can be improved to achieve full marks.

Here are guidelines to follow:
1. The advice you give to improve the Student Answer must be consistent with the advice in the Graded Answer.
2. Illustrate any advice given to the student with examples. These examples should be what the student wrote in their answer. If the example earns low marks, refine it to be worthy of higher marks.
3. Speak in second or third person, do not speak in first person.

Below are examples to guide you:

Question:
The Alliance System was the main cause of the First World War.' How far do you agree with this statement? Explain your answer.

Mark Scheme:
Level_4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question\n    - Answer demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance.\n    - Extends Level 3.\n  marks_awarded:\n    maximum: 16\n    minimum: 12\n  criteria:\n    - Complex explanation of stated factor and other factor(s)\n    - Accurate and detailed knowledge and understanding\n    - Complex, sustained line of reasoning with coherence and logical structure\n  progression:\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.\n  example_answer:\n    - ‘the Alliance system was one of the long term causes of the war in the wider context of European rivalry over empires. It did not in itself make war inevitable, but it created a framework for conflict. This was triggered in the short-term by the Austrian ultimatum to Serbia in 1914.’\n\nLevel_3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n    - Answer demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Answer demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance.\n    - Extends level 2 \n    - Answer may suggest that one reason has greater merit.\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  criteria:\n    - Developed explanation of the stated factor and other factor(s)\n    - Accurate knowledge and understanding\n    - Developed, sustained line of reasoning with coherence and logical structure\n  progression:\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding.\n  example_answer:\n    - “explaining the motives and rivalries that led to the Triple Alliance and the Triple Entente. This system resulted in Europe being divided into opposing armed camps and resulted in Germany worrying about ‘encirclement’.”\n\nLevel_2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n    - Answer demonstrates specific knowledge and understanding that is relevant to the question\n    - Answer demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant.\n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level.\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  criteria:\n    - Simple explanation of stated factor or other factor(s)\n    - Specific knowledge and understanding\n    - Simple, sustained line of reasoning with coherence and structure\n  progression:\n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding.\n  example_answer:\n    - ‘the main reason a war broke out was the rivalry that had been developing between Britain and Germany over the size of their empires and their armed forces. Both countries felt threatened by the development of the other and wanted an opportunity to show their superiority by winning a war.’\n\nLevel_1:\n  description: \n    - Basic explanation of one or more factors\n    - Answer demonstrates basic knowledge and understanding that is relevant to the question\n    - Answer demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit.\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  criteria:\n    - Basic explanation of one or more factors\n    - Basic knowledge and understanding\n    - Basic line of reasoning with coherence and structure\n  progression:\n    - Students recognise and provide a basic explanation of one or more factors.\n  example_answer:\n    - Students may offer a basic explanation of the stated factor, such as, the Alliance system brought all the European powers into a war together. They had promised to support each other if a war broke out.\n    - Students may offer basic explanations of other factor(s), for example, the assassination of Archduke Franz Ferdinand caused Austria to declare war and this became a World War.\n\nLevel_0:\n  description: \n    - Students either submit no evidence or fail to address the question\n  marks_awarded:\n    maximum: 0\n    minimum: 0\n  criteria:\n    - No evidence or fail to address the question\n  progression:\n    - N/A\n  example_answer:\n    - N/A

Student Answer:
I agree to some extent that the alliance system was the main cause of WW1. The 'Alliance system' was a factor in the causes of WW1 because, for example, Austria-Hungary had annexed Bosnia so Serbia was unhappy. Serbia had then got Russia involved which left Austria getting Germany's help because they had signed the 'blank cheque'. Another example was the failure of the Schlieffen plan as Britain and France were an alliance and they had step foot in Belgium (Britain and Belgium signed a treaty). In conclusion I agree to a certain extent that the alliance system was the main cause that that had led to WWI.

Graded Answer:
The student provides a basic explanation of the alliance system and its role in the causes of WW1. They mention the involvement of Austria-Hungary, Serbia, Russia, and Germany, as well as the failure of the Schlieffen Plan and the involvement of Britain and Belgium. However, the explanation lacks depth and detailed factual knowledge. The student does not fully develop their reasoning or provide a sustained line of argument.\n\nThe answer demonstrates basic knowledge and understanding of the alliance system and its impact on the outbreak of WW1. The line of reasoning is coherent but not well-developed or substantiated. The conclusion is brief and does not add much to the overall argument.\n\nTo reach a higher level, the student would need to provide a more detailed and developed explanation of the alliance system and other factors that contributed to the outbreak of WW1. They should also aim to create a more sustained and coherent line of reasoning, supported by accurate and detailed factual knowledge.\n\nOverall, the answer falls into Level 1 and should be awarded 4 marks.

Student Feedback:
You've made a good start by acknowledging the role of the alliance system, and it's clear that you have some understanding of how different nations were involved. Here are specific ways you can improve your response to achieve higher marks:
1. **Deepen Your Explanation of the Alliance System**: You mentioned that the alliance system was a factor in the causes of WWI, but it's important to elaborate on how it functioned. For instance, you could explain that the alliance system created a network of treaties that turned a local conflict into a larger war. Try mentioning how the Triple Alliance (Germany, Austria-Hungary, and Italy) and the Triple Entente (Britain, France, and Russia) were designed to provide mutual defense and how this heightened tensions. This added detail shows a deeper understanding of the alliances' implications.
2. **Provide More Detailed Factual Knowledge**: While you discuss the involvement of Austria-Hungary, Serbia, and Russia, try to enhance your argument with specific details. For example, you noted that Russia supported Serbia after Austria-Hungary annexed Bosnia. You could mention how this annexation caused unrest in the Balkans and how nationalist sentiments were further fueled, leading to the assassination of Archduke Franz Ferdinand. Including these details would strengthen your argument significantly and demonstrate your knowledge.
3. **Develop Your Reasoning and Structure**: You have a coherent line of reasoning, but it could be more sustained and persuasive. Instead of stating, "I agree to some extent," you could clarify whether you believe the alliance system was the primary cause or if other factors were equally important. For example, you might say, "While the alliance system contributed significantly to the outbreak of WWI by creating divided camps, factors such as nationalism and imperialism also played crucial roles." This type of reasoning will help you form a more complex judgment.
4. **Expand Your Conclusion**: Your conclusion is brief and doesn't add much depth to your argument. Instead of simply restating your opinion, summarize your main points more comprehensively and reinforce why the alliance system was significant in your analysis. You might say, "In conclusion, although the alliance system was a crucial factor leading to WWI by entangling nations in mutual defense obligations, it's important to recognize that other elements such as rising nationalism, imperial competition, and militarism also significantly contributed to the war's outbreak." This gives the reader a clearer sense of your overall argument.
5. **Use Clear Examples to Illustrate Points**: When discussing the Schlieffen Plan, consider elaborating on why it failed and its impact. You mentioned that "the failure of the Schlieffen Plan" occurred, but you could specify how this plan intended to quickly knock out France to avoid fighting a two-front war, leading to Germany’s inability to mobilize effectively and further entrenching Britain and France as allies against them.
By implementing these strategies, you'll enhance your answer by demonstrating a more complex understanding of the various causes of World War I, which is crucial for reaching the higher levels of the mark scheme. Keep up the hard work!

Question:
{question_text}

Mark Scheme:
{mark_scheme_text}

Student Answer:
{answer_text}

Graded Answer:
{llm_graded_answer}

Student Feedback:
""".strip()

In [166]:
prompt_dict = {"prompt": student_feedback_extract_prompt}

# Serializing json
json_object = json.dumps(prompt_dict, indent=4)
 
# Writing to sample.json
with open("student_feedback_prompt.json", "w") as outfile:
    outfile.write(json_object)


In [169]:
# Opening JSON file
# with open('student_feedback_prompt.json', 'r') as openfile:
 
#     # Reading from json file
#     json_object = json.load(openfile)["prompt"]

In [158]:
# TODO: Tightly constrain how feedback should be. Numbered list on advice, example revised answer.

#student_answer = full_marks_answer.copy()

if np.isnan( student_answer["llm_level_hitrate"] ):
    print("Student Feedback is only given for level hitrate.")

elif student_answer.get("llm_awarded_marks") == student_answer.get("total_marks"):
    chat_completion = "Good job, you achieved full marks."

else:
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are an expert teacher with a specialty in helping students how they can improve their answers to achieve the highest marks."
            },
            {
                "role": "user",
                "content": student_feedback_extract_prompt.format_map(student_answer),
            }
        ],
        model="gpt-4o-mini",
    )

In [159]:
student_feedback = chat_completion if isinstance(chat_completion, str) else chat_completion.choices[0].message.content
print(student_feedback)

You've made a solid effort in evaluating Sources B and C for their usefulness in understanding military technology and tactics during the First World War. It’s clear that you recognize the importance of both sources and their contributions. Here are specific suggestions to help you improve your response and aim for higher marks:

1. **Deepen the Evaluation of Source B**: 
   - You mentioned that Source B shows how the British army planned to use the 'creeping barrage'. To enhance this point, you could explain more about how this tactic was intended to work and its significance in changing military strategy. Rather than just stating that it "was supposed to work," it would be beneficial to clarify how “the creeping barrage aimed to provide artillery cover as troops advanced, which was intended to protect them from enemy fire while successfully capturing enemy trenches.” This level of detail showcases a more developed understanding of the content.

2. **Analyze the Context of Source B**:

# With Structured Outputs - Takes longer

In [147]:
student_feedback_extract_prompt = """
You will be given a Question, Mark Scheme, Student Answer and a Graded Answer on that answer.
The Graded Answer has feedback for how a student can improve their answer.
Your task is to reorientate and expand the Graded Answer to make the student the subject for the advice, you speak in a straightforward, clear and supportive way to the student and offer them specific advice on how their answer can be improved to achieve full marks.

Here are guidelines to follow:
1. The advice you give to improve the Student Answer must be consistent with the advice in the Graded Answer.
2. Illustrate any advice given to the student with examples. These examples should be what the student wrote in their answer. If the example earns low marks, refine it to be worthy of higher marks.
3. Speak in second or third person, do not speak in first person.

Below are examples to guide you:

Question:
The Alliance System was the main cause of the First World War.' How far do you agree with this statement? Explain your answer.

Mark Scheme:
Level_4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question\n    - Answer demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance.\n    - Extends Level 3.\n  marks_awarded:\n    maximum: 16\n    minimum: 12\n  criteria:\n    - Complex explanation of stated factor and other factor(s)\n    - Accurate and detailed knowledge and understanding\n    - Complex, sustained line of reasoning with coherence and logical structure\n  progression:\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.\n  example_answer:\n    - ‘the Alliance system was one of the long term causes of the war in the wider context of European rivalry over empires. It did not in itself make war inevitable, but it created a framework for conflict. This was triggered in the short-term by the Austrian ultimatum to Serbia in 1914.’\n\nLevel_3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n    - Answer demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Answer demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance.\n    - Extends level 2 \n    - Answer may suggest that one reason has greater merit.\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  criteria:\n    - Developed explanation of the stated factor and other factor(s)\n    - Accurate knowledge and understanding\n    - Developed, sustained line of reasoning with coherence and logical structure\n  progression:\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding.\n  example_answer:\n    - “explaining the motives and rivalries that led to the Triple Alliance and the Triple Entente. This system resulted in Europe being divided into opposing armed camps and resulted in Germany worrying about ‘encirclement’.”\n\nLevel_2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n    - Answer demonstrates specific knowledge and understanding that is relevant to the question\n    - Answer demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant.\n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level.\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  criteria:\n    - Simple explanation of stated factor or other factor(s)\n    - Specific knowledge and understanding\n    - Simple, sustained line of reasoning with coherence and structure\n  progression:\n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding.\n  example_answer:\n    - ‘the main reason a war broke out was the rivalry that had been developing between Britain and Germany over the size of their empires and their armed forces. Both countries felt threatened by the development of the other and wanted an opportunity to show their superiority by winning a war.’\n\nLevel_1:\n  description: \n    - Basic explanation of one or more factors\n    - Answer demonstrates basic knowledge and understanding that is relevant to the question\n    - Answer demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit.\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  criteria:\n    - Basic explanation of one or more factors\n    - Basic knowledge and understanding\n    - Basic line of reasoning with coherence and structure\n  progression:\n    - Students recognise and provide a basic explanation of one or more factors.\n  example_answer:\n    - Students may offer a basic explanation of the stated factor, such as, the Alliance system brought all the European powers into a war together. They had promised to support each other if a war broke out.\n    - Students may offer basic explanations of other factor(s), for example, the assassination of Archduke Franz Ferdinand caused Austria to declare war and this became a World War.\n\nLevel_0:\n  description: \n    - Students either submit no evidence or fail to address the question\n  marks_awarded:\n    maximum: 0\n    minimum: 0\n  criteria:\n    - No evidence or fail to address the question\n  progression:\n    - N/A\n  example_answer:\n    - N/A

Student Answer:
I agree to some extent that the alliance system was the main cause of WW1. The 'Alliance system' was a factor in the causes of WW1 because, for example, Austria-Hungary had annexed Bosnia so Serbia was unhappy. Serbia had then got Russia involved which left Austria getting Germany's help because they had signed the 'blank cheque'. Another example was the failure of the Schlieffen plan as Britain and France were an alliance and they had step foot in Belgium (Britain and Belgium signed a treaty). In conclusion I agree to a certain extent that the alliance system was the main cause that that had led to WWI.

Graded Answer:
The student provides a basic explanation of the alliance system and its role in the causes of WW1. They mention the involvement of Austria-Hungary, Serbia, Russia, and Germany, as well as the failure of the Schlieffen Plan and the involvement of Britain and Belgium. However, the explanation lacks depth and detailed factual knowledge. The student does not fully develop their reasoning or provide a sustained line of argument.\n\nThe answer demonstrates basic knowledge and understanding of the alliance system and its impact on the outbreak of WW1. The line of reasoning is coherent but not well-developed or substantiated. The conclusion is brief and does not add much to the overall argument.\n\nTo reach a higher level, the student would need to provide a more detailed and developed explanation of the alliance system and other factors that contributed to the outbreak of WW1. They should also aim to create a more sustained and coherent line of reasoning, supported by accurate and detailed factual knowledge.\n\nOverall, the answer falls into Level 1 and should be awarded 4 marks.

Student Feedback:
You've made a good start by acknowledging the role of the alliance system, and it's clear that you have some understanding of how different nations were involved. Here are specific ways you can improve your response to achieve higher marks:
1. **Deepen Your Explanation of the Alliance System**: You mentioned that the alliance system was a factor in the causes of WWI, but it's important to elaborate on how it functioned. For instance, you could explain that the alliance system created a network of treaties that turned a local conflict into a larger war. Try mentioning how the Triple Alliance (Germany, Austria-Hungary, and Italy) and the Triple Entente (Britain, France, and Russia) were designed to provide mutual defense and how this heightened tensions. This added detail shows a deeper understanding of the alliances' implications.
2. **Provide More Detailed Factual Knowledge**: While you discuss the involvement of Austria-Hungary, Serbia, and Russia, try to enhance your argument with specific details. For example, you noted that Russia supported Serbia after Austria-Hungary annexed Bosnia. You could mention how this annexation caused unrest in the Balkans and how nationalist sentiments were further fueled, leading to the assassination of Archduke Franz Ferdinand. Including these details would strengthen your argument significantly and demonstrate your knowledge.
3. **Develop Your Reasoning and Structure**: You have a coherent line of reasoning, but it could be more sustained and persuasive. Instead of stating, "I agree to some extent," you could clarify whether you believe the alliance system was the primary cause or if other factors were equally important. For example, you might say, "While the alliance system contributed significantly to the outbreak of WWI by creating divided camps, factors such as nationalism and imperialism also played crucial roles." This type of reasoning will help you form a more complex judgment.
4. **Expand Your Conclusion**: Your conclusion is brief and doesn't add much depth to your argument. Instead of simply restating your opinion, summarize your main points more comprehensively and reinforce why the alliance system was significant in your analysis. You might say, "In conclusion, although the alliance system was a crucial factor leading to WWI by entangling nations in mutual defense obligations, it's important to recognize that other elements such as rising nationalism, imperial competition, and militarism also significantly contributed to the war's outbreak." This gives the reader a clearer sense of your overall argument.
5. **Use Clear Examples to Illustrate Points**: When discussing the Schlieffen Plan, consider elaborating on why it failed and its impact. You mentioned that "the failure of the Schlieffen Plan" occurred, but you could specify how this plan intended to quickly knock out France to avoid fighting a two-front war, leading to Germany’s inability to mobilize effectively and further entrenching Britain and France as allies against them.
By implementing these strategies, you'll enhance your answer by demonstrating a more complex understanding of the various causes of World War I, which is crucial for reaching the higher levels of the mark scheme. Keep up the hard work!

Question:
{question_text}

Mark Scheme:
{mark_scheme_text}

Student Answer:
{answer_text}

Graded Answer:
{llm_graded_answer}

Student Feedback:
"""

In [150]:
class NumberedListItem(BaseModel):
    """Represents an individual item in the numbered list."""
    number: int
    instruction: str

class StudentFeedback(BaseModel):
    answer_feedback: str
    advice_to_improve_answer: List[NumberedListItem] = Field(default_factory=list)
    revised_answer: str

chat_completion = client.beta.chat.completions.parse(
    messages=[
        {
            "role": "system",
            "content": "You are an expert teacher with a specialty in helping students how they can improve their answers to achieve the highest marks."
        },
        {
            "role": "user",
            "content": student_feedback_extract_prompt.format_map(student_answer),
        }
    ],
    response_format=StudentFeedback,
    model="gpt-4o-mini",
)

student_feedback = chat_completion.choices[0].message.parsed.model_dump()
student_feedback

{'feedback_on_answer': "The student has made a commendable effort in analyzing the sources, demonstrating an understanding of the usefulness of both sources to historians studying military technology and tactics during the First World War. The references to 'creeping barrage' and personal accounts from the battlefield show a basic grasp of the content. However, the analysis could be strengthened significantly to achieve higher marks.",
 'advice_to_improve_answer': [{'number': 1,
   'value': "**Deepen the Evaluation of Source B's Utility**: While the student mentions the 'creeping barrage,' it's essential to elaborate on how this tactic aimed to help break the stalemate on the Western Front. For example, the student could explain that the creeping barrage was intended to provide cover for advancing infantry, illustrating how it represented a shift in military strategy. Additionally, consider mentioning the date and context of Source B to assess its reliability—was it written when the ef

In [146]:
print("LLM_Graded_Answer:\n", student_answer["llm_graded_answer"] )
print("_"*40)
print("Feedback on Student Answer:\n", student_feedback.get("feedback_on_answer"))

LLM_Graded_Answer:
 The student demonstrates a basic understanding of both sources and provides some analysis of their content. For Source B, the student identifies the use of the artillery barrage and explains its significance in military tactics, mentioning the concept of the 'creeping barrage'. They also touch on the historical context by noting that the initial use of this tactic resulted in friendly fire incidents. However, the evaluation of the source's utility is not fully developed, and there is no mention of the provenance of the source, which is necessary for higher-level responses.

For Source C, the student recognizes the value of a first-hand account from a tank commander and mentions the credibility of the source due to the author's direct experience and bravery award. They also highlight the introduction of the 'Whippet' tanks and their impact on the battlefield. However, the analysis lacks depth, and there is no detailed evaluation of the content or provenance.

To reac